In [1]:
# Estos dos comandos evitan que haya que hacer reload cada vez que se modifica un paquete
%load_ext autoreload
%autoreload 2

# Ejercicio de clasificación de texto

Naive Bayes es una técnica estadística que consiste en repetir el método anterior en problemas cuyos sucesos no son independientes, pero suponiendo independencia.
A lo largo de este trabajo desarrollarán un modelo de Naive Bayes para el problema de clasificación de artículos periodístios.En este caso podemos estimar la probabilidad de ocurrencia de cada palabra según la categoría a la que pertenece el artículo.

## Dataset


El primer paso es obtener el dataset que vamos a utilizar. El dataset a utilizar es el de TwentyNewsGroup(TNG) que está disponible en sklearn.

Se puede encontrar más información del dataset en la documentación de scikit-learn.

In [2]:
from sklearn.datasets import fetch_20newsgroups
from helper import autosave


@autosave('twenty-md={metadata}-{subset}')
def get_20newsgroup(subset='train', metadata=False):
    if metadata:
        return fetch_20newsgroups(subset=subset, shuffle=True)
    else:
        return fetch_20newsgroups(subset=subset, remove=('headers', 'footers', 'quotes'), shuffle=True)

In [3]:
#Loading the data set - training data.

twenty_train = get_20newsgroup(subset='train', metadata=False)

El siguiente paso es analizar el contenido del dataset, como por ejemplo la cantidad de artículos, la cantidad de clases, etc.

Preguntas:

1) ¿Cuántos articulos tiene el dataset?

In [4]:
len(twenty_train.data)

11314

2) ¿Cuántas clases tiene el dataset?

In [5]:
len(twenty_train.target_names)

20

3) ¿Es un dataset balanceado?

In [6]:
import numpy as np
_, counts = np.unique(twenty_train.target, return_counts=True)
if len(set(counts)) == 1:
    print('El dataset está balanceado')
else:    
    print('El dataset no está balanceado')

El dataset no está balanceado


4) ¿Cuál es la probabilidad a priori de la clase 5? A que corresponde esta clase?

In [7]:
priori5 = counts[5]/sum(counts)
print(f'La clase 5 ({twenty_train.target_names[5]}) tiene probabilidad a priori {priori5:3}')

La clase 5 (comp.windows.x) tiene probabilidad a priori 0.05241293972070002


5) ¿Cuál es la clase con mayor probabilidad a priori?

In [8]:
max_class = np.argmax(counts)
print(f'La clase {max_class} ({twenty_train.target_names[max_class]}) tiene la máxima probabilidad a priori, {counts[max_class]/sum(counts):.2}')

La clase 10 (rec.sport.hockey) tiene la máxima probabilidad a priori, 0.053


## Preprocesamiento

Para facilitar la comprensión de los algoritmos de preprocesamiento, se aplican primero a un solo artículo.


Mas info en:
http://text-processing.com/demo/stem/

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from helper import print_list


nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stemmer = PorterStemmer()

article = twenty_train.data[0]
article

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rochi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

- **Tokenization (nltk):**

In [10]:
tok = word_tokenize(article)
print_list(tok)
# print feo porque me ponia enters y quedaba muy largo

['I', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'was', 'a', '2-door', 'sports', 'car', ',', 'looked', 'to', 'be', 'from', 'the', 'late', '60s/', 'early', '70s', '.', 'It', 'was', 'called', 'a', 'Bricklin', '.', 'The', 'doors', 'were', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'is', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'of', 'production', ',', 'where', 'this', 'car', 'is', 'made', ',', 'history', ',', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', ',', 'please', 'e-mail', '.']
Palabras totales: 106
Palabras distintas: 71


- **Lemmatization (nltk):**

In [11]:
lemmatizer = WordNetLemmatizer()
lem=[lemmatizer.lemmatize(x, pos='v') for x in tok]
print_list(lem)

['I', 'be', 'wonder', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'I', 'saw', 'the', 'other', 'day', '.', 'It', 'be', 'a', '2-door', 'sport', 'car', ',', 'look', 'to', 'be', 'from', 'the', 'late', '60s/', 'early', '70s', '.', 'It', 'be', 'call', 'a', 'Bricklin', '.', 'The', 'doors', 'be', 'really', 'small', '.', 'In', 'addition', ',', 'the', 'front', 'bumper', 'be', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', '.', 'This', 'be', 'all', 'I', 'know', '.', 'If', 'anyone', 'can', 'tellme', 'a', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'of', 'production', ',', 'where', 'this', 'car', 'be', 'make', ',', 'history', ',', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'look', 'car', ',', 'please', 'e-mail', '.']
Palabras totales: 106
Palabras distintas: 67


- **Stop Words (nltk):**


In [12]:
stop = [x for x in lem if x not in stopwords.words('english')]
print_list(stop)

['I', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'I', 'saw', 'day', '.', 'It', '2-door', 'sport', 'car', ',', 'look', 'late', '60s/', 'early', '70s', '.', 'It', 'call', 'Bricklin', '.', 'The', 'doors', 'really', 'small', '.', 'In', 'addition', ',', 'front', 'bumper', 'separate', 'rest', 'body', '.', 'This', 'I', 'know', '.', 'If', 'anyone', 'tellme', 'model', 'name', ',', 'engine', 'specs', ',', 'years', 'production', ',', 'car', 'make', ',', 'history', ',', 'whatever', 'info', 'funky', 'look', 'car', ',', 'please', 'e-mail', '.']
Palabras totales: 69
Palabras distintas: 48


In [13]:
print(len(stopwords.words('english')))

179


- **Stemming (nltk):**


In [14]:
stem = [stemmer.stem(x) for x in stop]
print_list(stem)

['I', 'wonder', 'anyon', 'could', 'enlighten', 'car', 'I', 'saw', 'day', '.', 'It', '2-door', 'sport', 'car', ',', 'look', 'late', '60s/', 'earli', '70', '.', 'It', 'call', 'bricklin', '.', 'the', 'door', 'realli', 'small', '.', 'In', 'addit', ',', 'front', 'bumper', 'separ', 'rest', 'bodi', '.', 'thi', 'I', 'know', '.', 'If', 'anyon', 'tellm', 'model', 'name', ',', 'engin', 'spec', ',', 'year', 'product', ',', 'car', 'make', ',', 'histori', ',', 'whatev', 'info', 'funki', 'look', 'car', ',', 'pleas', 'e-mail', '.']
Palabras totales: 69
Palabras distintas: 48


- **Filtrado de palabras:**


In [15]:
alpha = [x for x in stem if x.isalpha()]
print_list(alpha)

['I', 'wonder', 'anyon', 'could', 'enlighten', 'car', 'I', 'saw', 'day', 'It', 'sport', 'car', 'look', 'late', 'earli', 'It', 'call', 'bricklin', 'the', 'door', 'realli', 'small', 'In', 'addit', 'front', 'bumper', 'separ', 'rest', 'bodi', 'thi', 'I', 'know', 'If', 'anyon', 'tellm', 'model', 'name', 'engin', 'spec', 'year', 'product', 'car', 'make', 'histori', 'whatev', 'info', 'funki', 'look', 'car', 'pleas']
Palabras totales: 50
Palabras distintas: 42


### Preprocesamiento completo

Utilizar o no cada uno de los métodos vistos es una decisión que dependerá del caso particular de aplicación. Para este ejercicio vamos a considerar las siguientes combinaciones:

- Tokenización
- Tokenización, Lematización, Stemming.
- Tokenización, Stop Words.
- Tokenización, Lematización, Stop Words, Stemming.
- Tokenización, Lematización, Stop Words, Stemming, Filtrado.

In [16]:
def filter_article(article, filts):
    filts = filts.split()

    if 'lem' in filts:
        article = [lemmatizer.lemmatize(x,pos='v') for x in article]

    if 'stop' in filts:
        article = [x for x in article if x not in stopwords.words('english')]

    if 'stem' in filts:
        article = [stemmer.stem(x) for x in article]

    if 'filt' in filts:
        article = [x for x in article if x.isalpha()]
    
    return article


@autosave(fmt='{name}-{filts}')
def filter_articles(name, articles, filts):
    
    filtered_articles = []
    for data in articles:
        tok = word_tokenize(data)
        filtered_articles.append(filter_article(tok, filts))
            
    return filtered_articles

In [17]:
ans_fmt = """Preprocesamiento: {preproc}
Longitud del vocabulario: {vocab_len}
"""
preprocessing = ['tok', 'tok lem stem', 'tok stop', 'tok lem stop stem', 'tok lem stop stem filt']

    
for preproc in preprocessing:
    filtered_articles = filter_articles('train-nometadata', twenty_train.data, preproc)
    vocab = set([word for article in filtered_articles for word in article])
    print(ans_fmt.format(preproc=preproc, vocab_len=len(vocab)))

Preprocesamiento: tok
Longitud del vocabulario: 161698

Preprocesamiento: tok lem stem
Longitud del vocabulario: 126903

Preprocesamiento: tok stop
Longitud del vocabulario: 161533

Preprocesamiento: tok lem stop stem
Longitud del vocabulario: 126872

Preprocesamiento: tok lem stop stem filt
Longitud del vocabulario: 45315



Preguntas

- Cómo cambia el tamaño del vocabulario al agregar Lematización y Stemming?

Sólo con tokenización: 161698 palabras

Con lematización y stemming: 126903 palabras (21.5% menor, 34795 palabras menos).

El vocabulario se reduce no porque la cantidad total de palabras sea menor, sino porque palabras que originalmente eran distintas ahora son iguales (por ejemplo, "is" y "was" se convierten ambas en "be").

La reducción del 20% sugiere que de cada 5 palabras, dos son dos "versiones" de la misma, lo cual es razonable si se tiene en cuenta que es muy común usar una misma palabra en singular y plural en un mismo contexto ("car" / "cars"), adverbios y adjetivos con la misma raíz ("real" / "really"), pronombres en distintos casos ("I" / "me", etcétera (en español esto probablemente sería incluso más pronunciado, al haber mayor cantidad de declinaciones verbales distintas, y tener géneros para adjetivos y artículos).

- Cómo cambia el tamaño del vocabulario al Stop Words?

Sólo con tokenización: 161698 palabras

Con stop words: 161533 palabras (0.1% menor, 165 palabras menos)

El vocabulario se reduce porque se remueven las palabras contenidas en el conjunto de stop words. Es razonable entonces que la reducción del vocabulario sea menor de este caso, ya que como máximo se podrán remover tantas palabras como haya en la lista de stop words (en este caso, 179).

- Analice muy brevemente ventajas y desventajas del tamaño del dataset en cada caso.

Para el caso de stopwords, es útil porque es razonable pensar que ese conjunto de palabras estará presente en todos los artículos, y por lo tanto no aportará demasiada información. Por ejemplo, una palabra como "car", "player" o "God" nos da más información sobre de qué se está hablando que "in", "no" o "is".

Sin embargo, también pueden imaginarse casos donde esto no sea cierto. Por ejemplo, si una clase se caracteriza por narraciones en primera persona, mientras que los demás suelen ser más impersonales, se estaría ignorando la información que aportan palabras como "I", "me", etc. Lo mismo puede ser cierto de segunda persona, o tercera persona "he/she/they" (en artículos que hablan de personas) en lugar de cosas ("it").

En cuanto a la lematización y stemming, puede hacerse un análisis muy similar. En muchos casos, es muy útil combinar las probabilidades de palabras similares: en un artículo de deportes, la palabra "partido" o "partidos" apuntan a un contexto muy similar, así como hablar de un evento que sucederá en el futuro, anticipando y especulando ("if they win"), o haciendo un racconto de eventos del pasado ("after they won") no es particularmente relevante.

Sin embargo, nuevamente pueden pensarse casos donde esto no será cierto: puede que una clase se caracterice por hablar sobre eventos del pasado, mientras que otra se dedique más a explicaciones en presente, y ese matiz se pierde.

## Vectorización de texto

- **Obtención del vocabulario y obtención de la probabilidad**

Como se vió en clase, los vectorizadores cuentan con dos parámetros de ajuste.

- max_df: le asignamos una maxima frecuencia de aparición, eliminando las palabras comunes que no aportan información.

- min_df: le asignamos la minima cantidad de veces que tiene que aparecer una palabra.


In [18]:
@autosave(fmt='{name}-joined-{filts}')
def get_filtered_joined_articles(name, articles, filts):
    articles = filter_articles(name, articles, filts)
    for i in range(len(articles)):
        articles[i] = ' '.join(articles[i])
        
    return articles

In [122]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
import pandas as pd
import os.path


OUT_FILE = 'out-nometadata5.csv'

vectorizers = [CountVectorizer, TfidfVectorizer]
classifiers = [MultinomialNB, BernoulliNB]
max_dfs = [0.005,0.01, 0.025, 0.04, .05, 0.1]
min_dfs = [1, 2, 5, 10]

results = []

import pandas as pd


total_its = len(vectorizers)*len(classifiers)*len(min_dfs)*len(max_dfs)*len(preprocessing)
i = 1

if os.path.isfile(OUT_FILE):
    results = pd.read_csv(OUT_FILE)
else:
    results = []

    for filts in preprocessing:
        data = get_filtered_joined_articles(name='train-nometadata', filts=filts) 
        
        for max_df in max_dfs:
            for min_df in min_dfs:
                
                for Vectorizer in vectorizers:
                    count_vect = Vectorizer(max_df=max_df, min_df=min_df)
                    raw_data = count_vect.fit_transform(data) 

                    for Classifier in classifiers:
                        print(f'Computing: F={filts}, M={max_df}, m={min_df}, V={Vectorizer.__name__}, C={Classifier.__name__} ({i}/{total_its})...')
                        i += 1
                        
                        clf = Classifier()
                        clf.fit(raw_data, twenty_train.target)
                        score = clf.score(raw_data, twenty_train.target)

                        results.append({
                            'max_df': max_df,
                            'min_df': min_df,
                            'filts': filts,
                            'vectorizer': Vectorizer.__name__,
                            'model': Classifier.__name__,
                            'score': score
                        })
                        
                        print(f'Score: {score:.5%}')
                        print(50 * '-')
    results = pd.DataFrame(results)
    results.to_csv(OUT_FILE, index=False)

In [123]:
results.sort_values(by=['score'], ascending=False, inplace=True)

In [124]:
results[results.model=='MultinomialNB'].sort_values(by='score', ascending=True).head()

,max_df,min_df,filts,vectorizer,model,score
240,0.95,1,tok lem stem,CountVectorizer,MultinomialNB,0.723528
224,0.80,1,tok lem stem,CountVectorizer,MultinomialNB,0.734135
208,0.75,1,tok lem stem,CountVectorizer,MultinomialNB,0.734135
112,0.95,1,tok,CountVectorizer,MultinomialNB,0.745183
80,0.75,1,tok,CountVectorizer,MultinomialNB,0.751635


In [125]:
results[results.model=='BernoulliNB'].head()

,max_df,min_df,filts,vectorizer,model,score
523,0.05,5,tok lem stop stem filt,TfidfVectorizer,BernoulliNB,0.682871
521,0.05,5,tok lem stop stem filt,CountVectorizer,BernoulliNB,0.682871
525,0.05,10,tok lem stop stem filt,CountVectorizer,BernoulliNB,0.678451
527,0.05,10,tok lem stop stem filt,TfidfVectorizer,BernoulliNB,0.678451
13,0.05,10,tok,CountVectorizer,BernoulliNB,0.678363


In [126]:
len(results)

640

In [127]:
best = results['score'].max()
best_filt = results['score']==best
best_hypers = results[best_filt]
best_hypers

,max_df,min_df,filts,vectorizer,model,score
2,0.05,1,tok,TfidfVectorizer,MultinomialNB,0.884568


In [128]:
best_hyper = best_hypers.iloc[0]

## Entrenamiento del modelo

Primero deben separar correctamente el dataset para hacer validación del modelo.

Y luego deben entrenar el modelo de NaiveBayes con el dataset de train.

Deben utilizar un modelo de NaiveBayes Multinomial y de Bernoulli. Ambos modelos estan disponibles en sklearn.

Finalmente comprobar el accuracy en train.

Preguntas

- Con que combinación de preprocesamiento obtuvo los mejores resultados? Explique por qué cree que fue así.

- Con que modelo obtuvo los mejores resultados? Explique por qué cree que fue así.

## Performance de los modelos

En el caso anterior, para medir la cantidad de artículos clasificados correctamente se utilizó el mismo subconjunto del dataset que se utilizó para entrenar.

Esta medida no es una medida del todo útil, ya que lo que interesa de un clasificador es su capacidad de clasificación de datos que no fueron utilizados para entrenar. Es por eso que se pide, para el clasificador entrenado con el subconjunto de training, cual es el porcentaje de artículos del subconjunto de testing clasificados correctamente. Comparar con el porcentaje anterior y explicar las diferencias.

Finalmente deben observar las diferencias y extraer conclusiones en base al accuracy obtenido, el preprocesamiento y vectorización utilizado y el modelo, para cada combinación de posibilidades.

In [129]:
#Loading the data set - training data.
twenty_test = get_20newsgroup(subset='test', metadata=False)

In [130]:
train_data = get_filtered_joined_articles(name='train-nometadata', 
                                        articles=twenty_train.data,
                                        filts=best_hyper.filts)
test_data = get_filtered_joined_articles(name='test-nometadata', 
                                        articles=twenty_test.data,
                                        filts=best_hyper.filts)

In [131]:
count_vect = eval(best_hyper.vectorizer)(max_df=best_hyper.max_df, min_df=best_hyper.min_df)
raw_data_train = count_vect.fit_transform(train_data) 
raw_data_test = count_vect.transform(test_data) 

In [132]:
clf = eval(best_hyper.model)()
clf.fit(raw_data_train, twenty_train.target)
score = clf.score(raw_data_test, twenty_test.target)
score

0.6801646309081253

In [133]:
print_list(list(count_vect.vocabulary_.keys())[:200])

['wondering', 'enlighten', 'car', 'saw', 'door', 'sports', 'looked', 'late', '60s', 'early', '70s', 'bricklin', 'doors', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'tellme', 'model', 'engine', 'specs', 'production', 'history', 'whatever', 'info', 'funky', 'fair', 'brave', 'souls', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'send', 'brief', 'message', 'detailing', 'procedure', 'top', 'speed', 'attained', 'cpu', 'rated', 'add', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'per', 'floppy', 'disk', 'functionality', '800', 'floppies', 'especially', 'requested', 'summarizing', 'next', 'days', 'network', 'knowledge', 'base', 'upgrade', 'answered', 'folks', 'mac', 'plus', 'finally', 'gave', 'ghost', 'weekend', 'starting', 'life', '512k', '1985', 'sooo', 'market', 'machine', 'sooner', 'intended', 'picking', 'powerbook', '160', '180', 'bunch', 'questions', 'hopefully', 'somebody', 'answer', 'anybody', 'dirt', 'round', 'introductio

In [134]:
twenty_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [135]:
eval(best_hyper.vectorizer)(max_df=best_hyper.max_df, min_df=best_hyper.min_df)

TfidfVectorizer(max_df=0.05)

Preguntas

- El accuracy en el dataset de test es mayor o menor que en train? Explique por qué.